# all rules - new algo

In [47]:
import os
import json
from collections import defaultdict as ddict
order_l = ["conclusion_first","premise_first"]
available_models = ["gpt"]
concat_file_name = "concat.json"
max_node_file_name = "max_node_len.json"
tail_head_list_template = ["all_tail_{rule_index}_nodelen_max","all_head_{rule_index}_nodelen_max"]
all_concat_data = ddict(lambda : ddict(lambda : ddict( lambda : ddict(dict))))

def assert_nozero(l):
	if len(l) != 0:
		return True
	else:
		return False


In [48]:
person_rule_keys = [f"rule{i}" for i in range(65)]
object_rule_keys = [f"rule{i}" for i in range(65, 201)]

In [49]:
print(person_rule_keys)
print(object_rule_keys)

['rule0', 'rule1', 'rule2', 'rule3', 'rule4', 'rule5', 'rule6', 'rule7', 'rule8', 'rule9', 'rule10', 'rule11', 'rule12', 'rule13', 'rule14', 'rule15', 'rule16', 'rule17', 'rule18', 'rule19', 'rule20', 'rule21', 'rule22', 'rule23', 'rule24', 'rule25', 'rule26', 'rule27', 'rule28', 'rule29', 'rule30', 'rule31', 'rule32', 'rule33', 'rule34', 'rule35', 'rule36', 'rule37', 'rule38', 'rule39', 'rule40', 'rule41', 'rule42', 'rule43', 'rule44', 'rule45', 'rule46', 'rule47', 'rule48', 'rule49', 'rule50', 'rule51', 'rule52', 'rule53', 'rule54', 'rule55', 'rule56', 'rule57', 'rule58', 'rule59', 'rule60', 'rule61', 'rule62', 'rule63', 'rule64']
['rule65', 'rule66', 'rule67', 'rule68', 'rule69', 'rule70', 'rule71', 'rule72', 'rule73', 'rule74', 'rule75', 'rule76', 'rule77', 'rule78', 'rule79', 'rule80', 'rule81', 'rule82', 'rule83', 'rule84', 'rule85', 'rule86', 'rule87', 'rule88', 'rule89', 'rule90', 'rule91', 'rule92', 'rule93', 'rule94', 'rule95', 'rule96', 'rule97', 'rule98', 'rule99', 'rule100

In [50]:
path_search = "path_to_likelihood_results_of_link"
path_baseline_gpt35 = "path_to_likelihood_results_of_chatgpt"
path_baseline_gpt4 = "path_to_likelihood_results_of_gpt4"

is_search_or_baseline_list = ["LINK","ChatGPT","GPT4"]
search_or_baseline_path = [path_search,path_baseline_gpt35,path_baseline_gpt4]


for index,path in enumerate(search_or_baseline_path):

	is_search_or_baseline = is_search_or_baseline_list[index]

	for order in order_l:
		concat_file = os.path.join(os.path.join(path,order),concat_file_name)
		max_node_file = os.path.join(os.path.join(path,order),max_node_file_name)
		if not os.path.exists(max_node_file):
			continue
		with open(max_node_file) as f:
			rule_keys = list(json.load(f).keys())
		# edit rule types here!
		# rule_keys = list(set(rule_keys) & set(person_rule_keys))
		with open(concat_file) as f:
			concat_data = json.load(f)

		for rule_key in rule_keys:
			tail_head_list = [_.format(rule_index = rule_key) for _ in tail_head_list_template]
			try:
				assert assert_nozero(concat_data[available_models[0]][tail_head_list[0]]),"break"
				assert assert_nozero(concat_data[available_models[0]][tail_head_list[1]]),"break"
			except:
				print(rule_key)
				print("skip")
				continue
			all_concat_data[is_search_or_baseline][rule_key]["tail"][available_models[0]] = concat_data[available_models[0]][tail_head_list[0]]
			all_concat_data[is_search_or_baseline][rule_key]["head"][available_models[0]] = concat_data[available_models[0]][tail_head_list[1]]


In [51]:
import numpy as np
def calculate_delta(l1,l2):
	# head - tail
	mean_l1 = np.mean(l1)
	mean_l2 = np.mean(l2)
	return mean_l1 - mean_l2
from collections import defaultdict as ddict



def delta_for_setting(delta_all, method,model):
	x = []
	rules = []
	for rule in delta_all[method]:
		rules.append(rule)
		x.append(delta_all[method][rule][model]["delta"])
	return x,rules

In [52]:
delta_all = ddict( lambda : ddict ( lambda :ddict( lambda :ddict( dict))))

if 1:
	x = [list(all_concat_data[_method].keys()) for _method in all_concat_data]
	availiable_rules = set([_y for y in x for _y in y])
	for y in x:
		availiable_rules = availiable_rules.intersection(y)
	availiable_rules = list(availiable_rules)
	availiable_rules = sorted(availiable_rules, key= lambda x: int(x.replace("rule","")))
	for method in all_concat_data:
		for rule in availiable_rules:
			delta_all[method][rule]["gpt"]["head"] = \
			np.mean(all_concat_data[method][rule]["head"]["gpt"])
			delta_all[method][rule]["gpt"]["head_len"] = \
			len(all_concat_data[method][rule]["head"]["gpt"])
			delta_all[method][rule]["gpt"]["tail"] = \
			np.mean(all_concat_data[method][rule]["tail"]["gpt"])
			delta_all[method][rule]["gpt"]["tail_len"] = \
			len(all_concat_data[method][rule]["tail"]["gpt"])
			delta_all[method][rule]["gpt"]["delta"] = \
			calculate_delta(all_concat_data[method][rule]["head"]["gpt"],all_concat_data[method][rule]["tail"]["gpt"])



In [53]:
import plotly.express as px
import pandas as pd

# rule_name = {
#     "rule_person":"Person related rules",
#     "rule_object":"Object related rules"
# }
# rule_name = "Person related rules"
# rule_name = "Object related rules"
rule_name = "All rules"
llh_name = {
    "llama":"Llama-7B",
    "gpt":"text-davinci-003"
}


model = "gpt"


data = {}
for method in is_search_or_baseline_list:
        data[method] ,rules = delta_for_setting(delta_all,method,model)
        data["rules"] = rules

df = pd.DataFrame(data)

print(f'LINK: {df["LINK"].mean()}')
print(f'ChatGPT: {df["ChatGPT"].mean()}')
print(f'GPT4: {df["GPT4"].mean()}')
fig = px.line(df, x='rules', y=["LINK","ChatGPT","GPT4"], title=f'<b>Delta</b> value for <b>{rule_name}</b> between Head and Long-tail distribution over <b>{llh_name[model]}</b>')
fig.update_yaxes(title_text="<b>delta</b>")
fig.update_layout(legend_title_text='')
fig.update_layout(xaxis_tickangle=-90)
fig.update_layout(width=1800)
fig.update_layout(
    legend_font=dict(size=15)         
)
line = dict(
    type="line",
    xref="x", 
    yref="y",
    x0=-0.5,  # 开始位置，可以根据需要调整
    y0=0,     # 线的高度
    x1=200,   # 结束位置，可以根据需要调整
    y1=0,     # 线的高度
    line=dict(
        color="black",  # 线的颜色
        width=2        # 线的宽度
    )
)

# 更新图形布局，添加线
fig.update_layout(
    shapes=[line]
)
fig.update_yaxes(range=[-1, 2])
fig.show()
import plotly.io as pio
img_bytes = pio.to_image(fig, format="pdf", scale=7)
with open("delta_all_rules.pdf", "wb") as f:
    f.write(img_bytes)


LINK: 0.47591313184193657
ChatGPT: -0.1392557447303751
GPT4: -0.02002096774176598
